In [ ]:
import sys
import cv2
import easygui
import numpy as np
sys.path.append('py_modules')
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from processing import ReadImage, WriteImage
from post_processing import Threshholding, CleanUp, build_sknw,  WritePoints2SHP, WritePolyline2SHP

In [ ]:
filename  = easygui.fileopenbox("select binary image file", "postProcs", filetypes= "*.jpg", multiple=False)
image = ReadImage(filename)
MASTER = np.zeros(image[0].shape, np.double)

## Threshholding

In [ ]:
def InteractThresh(thresh = 0.1, ksize = 5, alpha = 1.5):
    thresh_img = Threshholding(image[0], thresh, ksize, alpha, 0)
    
    x1, x2 = thresh_img.shape[:2] 
    MASTER[:x1, :x2] = thresh_img[:x1, :x2]
    
    f, ax1 = plt.subplots(nrows=1,figsize=(25,25))      
    ax1.imshow(MASTER, cmap="gray"); 
    ax1.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)

style = {'description_width': 'initial'}
widgets.interactive(InteractThresh, thresh = widgets.FloatSlider(description='min pixel value',style=style, value=0.1 ,min=0, max=1, step=0.01,continuous_update=False, layout=widgets.Layout(width='50%')),
                                   ksize = widgets.IntSlider(description='kernel size',style=style, value=3 ,min=1, max=100, step=2,continuous_update=False, layout=widgets.Layout(width='50%')),
                                   alpha = widgets.FloatSlider(description='contrast',style=style, value=1.5, min=1, max=100, step=0.1,continuous_update=False, layout=widgets.Layout(width='50%'))
                                   )   


## Clean


In [ ]:
def InteractCleanup(connectivity = 8, min_size = 10):
    clean_img = CleanUp(MASTER, connectivity, min_size)
    
    x1, x2 = clean_img.shape[:2] 
    MASTER[:x1, :x2] = clean_img[:x1, :x2]
    
    f, ax1 = plt.subplots(nrows=1,figsize=(25,25))      
    ax1.imshow(MASTER, cmap="gray"); 
    ax1.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)

style = {'description_width': 'initial'}
widgets.interactive(InteractCleanup,connectivity = widgets.IntSlider(description='connectivity',style=style, value=8 ,min=1, max=8, step=1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                   min_size = widgets.IntSlider(description='min cluster size',style=style, value=10 ,min=1, max=1000, step=1,continuous_update=False, layout=widgets.Layout(width='50%'))
                                   )   

## Build graph and write shapefiles

In [ ]:
graph = build_sknw(MASTER, multi=False, iso=True, ring=True, full=True)
WritePoints2SHP(graph, 'pt_test.shp', refWKT = None, tolerance=1e-3)
WritePolyline2SHP(graph, 'line_test.shp', refWKT = None, tolerance=1e-3)